In [9]:
import pandas as pd
import os
import re
import warnings

In [10]:
file_path = 'C:/Users/htet.ma\\Downloads\\Assembly Inspected'
file_names = os.listdir(file_path)

# all_sheets = {}
all_sheets_xls = []

for name in file_names:
    first_sheet = pd.read_excel(file_path + "\\" + str(name), sheet_name=0, header=0, skiprows=[1])
    # csv_filename = os.path.splitext(name)[0] + '.csv'  # Create the corresponding CSV filename
    # first_sheet.to_csv(csv_filename, index=False)  # Save the sheet as a CSV file
    all_sheets_xls.append(first_sheet)
#     all_sheets[name] = first_sheet

combined_xls = pd.concat(all_sheets_xls, ignore_index=True)

print(f'finished')

finished


In [11]:
# Dictionary

# exception: GIF-N180 and GIF-XP180N

# Refurbishment
refurb = {'TJF': ['509A.21', '513E.01', '509A.26', '508I.52', '502H.14', '508I.44', '509A.14', '508I.53', '513A.01', '509A.18', '3000R.29', '509A.17'],
          'PJF': ['509A.21', '513E.01', '509A.26', '508I.52', '502H.14', '508I.44', '509A.14', '508I.53', '513A.01', '509A.18', '3000R.29', '509A.17'],
          'JF': ['509A.21', '513E.01', '509A.26', '508I.52', '502H.14', '508I.44', '509A.14', '508I.53', '513A.01', '509A.18', '3000R.29', '509A.17'],
          'GF': ['509A.21', '513E.01', '509A.26', '508I.52', '502H.14', '508I.44', '509A.14', '508I.53', '513A.01', '509A.18', '3000R.29', '509A.17'],
          'Everything': ['509A.21', '513E.01', '509A.26', '508I.52', '502H.14', '508I.44', '509A.14', '508I.53', '513A.01', '509A.18', '3000R.29', 'N/A']}

upd_check = {'CF-Q160DL': ['501K.04'], 'CF-Q160DI': ['501K.04'], 'CF-H180DL': ['501K.04'], 'CF-H180DI': ['501K.04'], 
             'CF-HQ190L': ['501K.04'], 'CF-HQ190I': ['501K.04'], 'CF-HQ1100DL': ['501K.04'], 'CF-HQ1100DI': ['501K.04'],
             'CF-EZ1500DL': ['501K.04'], 'CF-EZ1500DI': ['501K.04'], 'PCF-H190DL': ['501K.04'], 'PCF-H190DI': ['501K.04'], 
             'Everything': ['N/A']}

print(f'finished')

finished


In [12]:
def get_chapters(model_generation, dictionary_group, group_repair):
    chapters = []
    
    if model_generation in dictionary_group.keys():
        for value in dictionary_group[model_generation]:
            if value in group_repair['Work Instruction Document Number '].values:
                chapters.append(0)
            else:
                chapters.append(value)
    else:
        for value in dictionary_group['Everything']:
            if value in group_repair['Work Instruction Document Number '].values:
                chapters.append(0)
            else:
                chapters.append(value)
            
    return chapters

In [13]:
def get_model(model_name, method):
    if method == 'type': #GIF, CF, etc.
        if '-' in model_name:
            split = model_name.split('-')
            return split[0]
        else:
            return model_name
    elif method == 'generation': # 170, 190
        for index, char in enumerate(model_name):
            if char == '-':
                split = model_name.split('-')
                split_num = re.findall(r'\d+', split[1])
                if split_num:
                    return split_num[0]
                else:
                    return split[1]
    elif method == 'entire model': #GIF-H190DL
        return model_name
    else:
        return print('Method is not written')

In [14]:
#Refurb
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    filtering = combined_xls[(combined_xls['OperationName'] == 'Assembly') & (combined_xls['LaborCode'].isin(['28A', '28D', '28F']))]

    groupping = filtering.groupby(['ModelFamily', 'ModelNo', 'LaborCode'])

    repair_checks = pd.DataFrame(columns=['Family', 'Model', 'LaborCode', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'])

    for group_key, group_repair in groupping:
    #     print("Group Key:", group_key)
    #     print(group_repair)
    #     print()

        model_number = get_model(group_key[1], 'type')

        missing_chapters = get_chapters(model_number, refurb, group_repair)

        together = [group_key[0]] + [group_key[1]] + [group_key[2]] + missing_chapters
        repair_checks = repair_checks.append(pd.Series(together, index=repair_checks.columns), ignore_index=True)

    print(f'finished')

finished


In [15]:
#UPD
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    filtering = combined_xls[(combined_xls['OperationName'] == 'Assembly') & (combined_xls['LaborCode'].isin(['28A', '28D', '28F']))]

    groupping = filtering.groupby(['ModelFamily', 'ModelNo', 'LaborCode'])

    upd_checks = pd.DataFrame(columns=['Family', 'Model', 'LaborCode', '1'])

    for group_key, group_repair in groupping:

        model_number = get_model(group_key[1], 'entire model')

        missing_chapters = get_chapters(model_number, upd_check, group_repair)

        together = [group_key[0]] + [group_key[1]] + [group_key[2]] + missing_chapters
        upd_checks = upd_checks.append(pd.Series(together, index=upd_checks.columns), ignore_index=True)

    print(f'finished')

finished


In [16]:
repair_final = pd.merge(repair_checks, upd_checks, on=['Family', 'Model', 'LaborCode'], how='outer', indicator=True)

output_file_path = 'C:/Users/htet.ma\\Downloads\\refurbishment_final.xlsx'
repair_final.to_excel(output_file_path, index=False)

print(f'finished')

finished
